In [12]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from ipywidgets import interact 

try:
    from colorspacious import cspace_converter
except ModuleNotFoundError:
    !pip install colorspacious
    from colorspacious import cspace_converter

np.random.seed(1234)

img_width = 400
ls  = np.linspace(0, 10, img_width)
x,y = np.meshgrid(ls, ls)

noise = 0.3
data = np.sin(x) * np.cos(y)

def gray_cm(cmapname):
    x = np.linspace(0.0, 1.0, 100)
    rgb = cm.get_cmap(plt.get_cmap(cmapname))(x)[np.newaxis, :, :3]
    lab = cspace_converter("sRGB1", "CAM02-UCS")(rgb)
    L = lab[0, :, 0]
    L = np.float32(np.vstack((L, L, L))).transpose() / 100

    return ListedColormap(L)

    
@interact(colormap=['viridis', 'jet', 'gray','PiYG','flag','Set1', 'turbo', 'prism'], gray=False, noise=(0, 0.4, 0.05))
def plot_data(colormap='viridis', gray=False, noise=0.3):
    fig, ax = plt.subplots(figsize=(8,8))
    
    if gray:
        colormap = gray_cm(colormap)
    
    im_data = data + noise * np.random.rand(img_width, img_width)
    
    im = ax.imshow(im_data, cmap=colormap)
    ax.invert_yaxis()
    ax.set_title("Some data")
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax)
    cbar.set_label('Value')
        
    plt.tight_layout()
    plt.show()

interactive(children=(Dropdown(description='colormap', options=('viridis', 'jet', 'gray', 'PiYG', 'flag', 'Set…

In [15]:
!pip install session_info
import session_info
session_info.show()

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8053 sha256=b18d657455a95d1f92896a842ddec469b3126bc34a2bdf23ec2b16abe808f84e
  Stored in directory: c:\users\alexander\appdata\local\pip\cache\wheels\bd\ad\14\6a42359351a18337a8683854cfbba99dd782271f2d1767f87f
Successfully built session-info
